# Objective:
1. We would like to predict, who are going to become as defaulter or deliquent customer
2. Defination: any customers, who are not paying the bills contionously for 180 days those customers are called as deliquent customers
3. data should have both acquistion file and performance file
4. curate the data, explore the data, treat the data
5. varaible creation,varaiable reduction 
6. model building and model validation
7. providing the recommendations

In [1]:
# import libraries into python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # visualization
import seaborn as sns # visulalization
from sklearn.model_selection import train_test_split # to split the data into training and validation
%matplotlib inline

In [2]:
col_acq = ['LoanID','Channel','SellerName','OrInterestRate','OrUnpaidPrinc','OrLoanTerm',
         'OrDate','FirstPayment','OrLTV','OrCLTV','NumBorrow','DTIRat','CreditScore',
        'FTHomeBuyer','LoanPurpose','PropertyType','NumUnits','OccStatus','PropertyState',
        'Zip','MortInsPerc','ProductType','CoCreditScore','MortInsType','RelMortInd']

col_per = ['LoanID','ReportingDate','Servicer','CurrInterestRate','CAUPB','LoanAge','MonthsToMaturity',
          'AdMonthsToMaturity','MaturityDate','MSA','CurDelStatus','ModFlag','ZeroBalCode','ZeroBalEffDate',
          'LastInstallDate','ForeclosureDate','DispositionDate','ForeclosureCost','RepairCost','AssetRecCost','MiscCostsPF',
           'ATFHP','NetSaleProceeds','CreditEnhProceeds','RPMWP','OtherForePro','NonInterestUPB','PricipleForgiven','RMWPF',
          'FPWA','ServicingIndicator']

In [3]:
acq07Q4=pd.read_csv("C:\\Users\\Koti\\Downloads\\Acquisition_2007Q4.txt",sep="|",names=col_acq)

per07Q4=pd.read_csv("C:\\Users\\Koti\\Downloads\\Performance_2007Q4.txt",sep="|",names=col_per, usecols=[0, 15])


In [4]:
print(acq07Q4.shape,per07Q4.shape)

(391209, 25) (20864257, 2)


In [6]:
per07Q4.tail()

LoanID ForeclosureDate
20864252  999999565669             NaN
20864253  999999565669             NaN
20864254  999999565669             NaN
20864255  999999565669             NaN
20864256  999999565669             NaN

In [7]:
per07Q4.drop_duplicates(subset='LoanID', keep='last', inplace=True)

In [8]:
per07Q4.shape

(391209, 2)

In [9]:
acq_perf=pd.merge(acq07Q4,per07Q4,on='LoanID')

In [10]:
acq_perf.shape

(391209, 26)

In [11]:
list(acq_perf.columns)

['LoanID',
 'Channel',
 'SellerName',
 'OrInterestRate',
 'OrUnpaidPrinc',
 'OrLoanTerm',
 'OrDate',
 'FirstPayment',
 'OrLTV',
 'OrCLTV',
 'NumBorrow',
 'DTIRat',
 'CreditScore',
 'FTHomeBuyer',
 'LoanPurpose',
 'PropertyType',
 'NumUnits',
 'OccStatus',
 'PropertyState',
 'Zip',
 'MortInsPerc',
 'ProductType',
 'CoCreditScore',
 'MortInsType',
 'RelMortInd',
 'ForeclosureDate']

In [12]:
acq_perf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 391209 entries, 0 to 391208
Data columns (total 26 columns):
LoanID             391209 non-null int64
Channel            391209 non-null object
SellerName         391209 non-null object
OrInterestRate     391208 non-null float64
OrUnpaidPrinc      391209 non-null int64
OrLoanTerm         391209 non-null int64
OrDate             391209 non-null object
FirstPayment       391209 non-null object
OrLTV              391209 non-null int64
OrCLTV             391177 non-null float64
NumBorrow          391203 non-null float64
DTIRat             379728 non-null float64
CreditScore        390583 non-null float64
FTHomeBuyer        391209 non-null object
LoanPurpose        391209 non-null object
PropertyType       391209 non-null object
NumUnits           391209 non-null int64
OccStatus          391209 non-null object
PropertyState      391209 non-null object
Zip                391209 non-null int64
MortInsPerc        84925 non-null float64
ProductT

In [13]:
acq_perf.rename(index=int, columns={"ForeclosureDate": 'Default'}, inplace=True)

# defination of defaulter:
1. if default is missing then those customers non defaulters
2. if default date is presented then those customers are called as defaulters

In [16]:
acq_perf['Default'].fillna(0, inplace=True)
acq_perf['Default'] =np.where(acq_perf['Default']==0,0,1)
acq_perf.loc[acq_perf['Default'] != 0, 'Default'] = 1

In [18]:
acq_perf["Default"].value_counts()/len(acq_perf)*100

0    90.996117
1     9.003883
Name: Default, dtype: float64

In [19]:
sns.countplot(acq_perf["Default"])

In [20]:
acq_perf.nunique()

LoanID            391209
Channel                3
SellerName            14
OrInterestRate       528
OrUnpaidPrinc        649
OrLoanTerm           140
OrDate                53
FirstPayment          53
OrLTV                 97
OrCLTV               120
NumBorrow              9
DTIRat                64
CreditScore          386
FTHomeBuyer            3
LoanPurpose            3
PropertyType           5
NumUnits               4
OccStatus              3
PropertyState         54
Zip                  893
MortInsPerc           18
ProductType            1
CoCreditScore        359
MortInsType            2
RelMortInd             2
Default                2
dtype: int64

In [21]:
acq_perf=acq_perf.drop(["ProductType","Zip","PropertyState","LoanID"],axis=1)

In [22]:
acq_perf.shape

(391209, 22)

In [ ]:
from sklearn.linear_model import LinearRegression  # this will help us to impute the numerical varaible missing values
from sklearn.linear_model import LogisticRegression # this will help us to impute the categorical varaible missing values
def fillnan(df):
    columns = df.columns[df.isnull().any()]
    for name in columns:
        y = df.loc[df[name].notnull(), name].values # the varaibles, which are having missing but consider only non missing rows of missing variables
        X = df.loc[df[name].notnull()].drop(columns, axis=1).values # non missing variables values are considering
        X_test = df.loc[df[name].isnull()].drop(columns, axis=1).values # missing rows 
        if df[name].dtypes == 'object':
            model = LogisticRegression()
            model.fit(X, y)
            df.loc[df[name].isnull(), name] = model.predict(X_test)
        else:
            model = LinearRegression()
            model.fit(X, y)
            df.loc[df[name].isnull(), name] = model.predict(X_test)
    return df